In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Filepaths

In [13]:
rootpath = '/media/sf_VBox_Shared/London/raw/first5/'
annotations_path = rootpath + 'tud_first5.csv'
wearcodes_path = rootpath + 'wearcodes.csv'

In [14]:
filepath = rootpath + 'output_first5/meta/csv/'

## Functions

In [31]:
def loadData(name):
    #try:
        name = name + '.RData.csv'
        filename = filepath + name
        data = pd.read_csv(filename, index_col='timestamp', parse_dates=True)
        return data
    #except:
        # print 'Fine not found: ',name
    #    return None

In [7]:
def parseDate(dateStr):
    try:
        return datetime.datetime.strptime(dateStr, '%d/%m/%Y %H:%M:%S')
    except:
        # hack -- 
        dateStr = dateStr.replace('24:','00:')
        dateStr = dateStr.replace('25:','01:')
        dateStr = dateStr.replace('27:','03:')
        return datetime.datetime.strptime(dateStr, '%d/%m/%Y %H:%M:%S')

def addAnnotations(data, fileAnnotations):
    for row in fileAnnotations[['ddate1', 'stime', 'etime', 'activity', 'filename']].as_matrix():
        rStart = parseDate(row[0].strip() + ' ' + row[1].strip())
        rEnd   = parseDate(row[0].strip() + ' ' + row[2].strip())
        data.loc[(data.index < rEnd) & (data.index > rStart), 'Activity'] = row[3]
        
    return data

## Load and preprocess annotations

In [15]:
annotations = pd.read_csv(annotations_path)
print(annotations.shape)
annotations.head()

(1440, 5)


,serflag,start_time,end_time,act,act_label
0,52920071_100,2015-03-01T04:00:00.000Z,2015-03-01T04:10:00.000Z,1.0,Sleeping and resting (including sick in bed)
1,52920071_100,2015-03-01T04:10:00.000Z,2015-03-01T04:20:00.000Z,1.0,Sleeping and resting (including sick in bed)
2,52920071_100,2015-03-01T04:20:00.000Z,2015-03-01T04:30:00.000Z,1.0,Sleeping and resting (including sick in bed)
3,52920071_100,2015-03-01T04:30:00.000Z,2015-03-01T04:40:00.000Z,1.0,Sleeping and resting (including sick in bed)
4,52920071_100,2015-03-01T04:40:00.000Z,2015-03-01T04:50:00.000Z,1.0,Sleeping and resting (including sick in bed)


In [16]:
wearcodes = pd.read_csv(wearcodes_path)
print(wearcodes.shape)
wearcodes.head()

(5, 8)


,Monitor,Day1,Day2,binFile,serflag,fileSize,day1Ready,day2Ready
0,16566,11/03/2015,15/03/2015,__016566_2015-03-24 14-52-30.bin,81510041_100,777664025,True,True
1,16580,05/03/2015,08/03/2015,__016580_2015-03-12 09-09-15.bin,102650051_100,522410444,True,True
2,18618,15/03/2015,16/03/2015,__018618_2015-03-20 11-15-03.bin,62980071_100,565693441,True,True
3,18635,01/03/2015,03/03/2015,__018635_2015-03-11 14-06-25.bin,52920071_100,692552249,True,True
4,18643,06/03/2015,07/03/2015,__018643_2015-03-13 12-19-47.bin,122650071_100,525756098,True,True


In [20]:
annotations_codes = pd.merge(annotations, wearcodes, on='serflag', how='left')
print(annotations_codes.shape)
annotations_codes.head()

(1440, 12)


,serflag,start_time,end_time,act,act_label,Monitor,Day1,Day2,binFile,fileSize,day1Ready,day2Ready
0,52920071_100,2015-03-01T04:00:00.000Z,2015-03-01T04:10:00.000Z,1.0,Sleeping and resting (including sick in bed),18635,01/03/2015,03/03/2015,__018635_2015-03-11 14-06-25.bin,692552249,True,True
1,52920071_100,2015-03-01T04:10:00.000Z,2015-03-01T04:20:00.000Z,1.0,Sleeping and resting (including sick in bed),18635,01/03/2015,03/03/2015,__018635_2015-03-11 14-06-25.bin,692552249,True,True
2,52920071_100,2015-03-01T04:20:00.000Z,2015-03-01T04:30:00.000Z,1.0,Sleeping and resting (including sick in bed),18635,01/03/2015,03/03/2015,__018635_2015-03-11 14-06-25.bin,692552249,True,True
3,52920071_100,2015-03-01T04:30:00.000Z,2015-03-01T04:40:00.000Z,1.0,Sleeping and resting (including sick in bed),18635,01/03/2015,03/03/2015,__018635_2015-03-11 14-06-25.bin,692552249,True,True
4,52920071_100,2015-03-01T04:40:00.000Z,2015-03-01T04:50:00.000Z,1.0,Sleeping and resting (including sick in bed),18635,01/03/2015,03/03/2015,__018635_2015-03-11 14-06-25.bin,692552249,True,True


## Load and preprocess data

In [33]:
byName = annotations_codes.groupby('binFile')
dfs = {}
for name, fileAnnotations in byName:
    # Load data
    df = loadData(name)
    
    if df is not None:    
        # Add annotations
        #df = addAnnotations(df, fileAnnotations)
        dfs[name] = df